In [1]:
#Import Library

import json
import random
import numpy as np
import tensorflow as tf
import nltk
nltk.download('punkt', download_dir='Bangkit/NLTK Path')
nltk.download('punkt_tab', download_dir='Bangkit/NLTK Path')
nltk.download('wordnet', download_dir='Bangkit/NLTK Path')
nltk.data.path.append('Bangkit/NLTK Path')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to Bangkit/NLTK Path...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to Bangkit/NLTK Path...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to Bangkit/NLTK Path...


In [2]:
#load intents json

data_file = open('augmentedDatasetCombined.json').read()
intents = json.loads(data_file)

words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [5]:
#Preprocessing Text

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(words), "unique lemmatized words")
print (len(classes), "classes", classes)

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

138 documents
83 unique lemmatized words
10 classes ['akibat_diabetes_tanpa_pengobatan', 'cara_mencegah_diabetes', 'cara_mengobati_diabetes', 'ciri_ciri_diabetes', 'fakta_unik_diabetes', 'gejala_diabetes', 'greetings', 'kondisi_kuku_diabetes', 'pengertian_diabetes', 'thankyou']


In [6]:
#Initializing Training Data

training = []
output_empty = [0] * len(classes)

for doc in documents:
    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create bag of words
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    # create output row
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # append to training data
    training.append([bag, output_row])

# shuffle training data
random.shuffle(training)

# separate features (X) and labels (Y)
train_x = np.array([np.array(i[0], dtype=np.float32) for i in training])
train_y = np.array([np.array(i[1], dtype=np.float32) for i in training])



In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 128 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

D:\Bangkit\ChatbotInVscode\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=16, verbose=1)
model.save('chatbot_modelV2.h5', hist)

print("model created")

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.0919 - loss: 2.3101
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1105 - loss: 2.2385
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2299 - loss: 2.1802
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2298 - loss: 2.1358
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3461 - loss: 2.0481
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3107 - loss: 1.9897
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3853 - loss: 1.9093
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4485 - loss: 1.7825
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3878 - loss: 1.7360
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5614 - loss: 1.6031
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6146 - loss: 1.3907
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6940 - lo

model created


In [11]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_modelV2.h5')

intents = json.loads(open('augmentedDatasetCombined.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:
#Function to clean user input

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

In [13]:
#Function for Bag of Wrds

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [14]:
#Function for Class Prediction

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

In [15]:
#Function to get chatbot response

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break

    return result

In [16]:
#Chatbot Function

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
while True:
    user_input = input()
    if user_input.lower == "exit" :
        break
    print(chatbot_response(user_input))



 hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 785ms/step
Hey! How can I assist you today?


 thanks


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
You're welcome!


 apa itu diabetes


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Diabetes adalah kondisi kronis di mana tubuh tidak dapat mengolah glukosa darah dengan baik karena kurangnya insulin atau resistensi terhadap insulin.


 kalau tidak diobati bagaimana?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Jika diabetes tidak diobati, dapat menyebabkan komplikasi serius seperti kerusakan saraf (neuropati), penyakit jantung, gagal ginjal, dan gangguan penglihatan bahkan kebutaan.
